# Build an e-commerce recommendation AI agents with ADK + Vector Search

Original Author(s): [Kaz Sato](https://github.com/kazunori279)

In this tutorial, we will explore how to build a simple multi-agent system for an e-commerce site, designed to offer the "Generative Recommendations" you find in the [Shopper's Concierge demo](https://www.youtube.com/watch?v=LwHPYyw7u6U). A step-by-step guide video is also available [here](https://youtu.be/07GX28rk7Yc).

### What is Generative Recommendation?

Generative Recommendation refers to the AI's ability to not only retrieve items directly matching a user's explicit search query but also to **intelligently infer, expand upon, or create new search queries and item suggestions based on a deeper understanding of the user's intent, external research, or contextual information.**

Here's a breakdown of its capabilities as demonstrated:

1.  **Understanding and Expanding User Intent:** Instead of simply taking a direct query, the system can interpret the underlying need. For instance, when a user asks for a "birthday present for 10 years old boy," the AI doesn't just search for those exact keywords.
2.  **Leveraging External Research (Google Search):** The "Generative Recommendation" process involves using tools like Google Search to perform market research. This allows the AI to understand what kind of items are generally purchased for a given intent (e.g., popular gifts for 10-year-old boys).
3.  **Generating New Queries:** Based on this research, the AI actively *generates* a list of more specific and diverse search queries. These generated queries go beyond the initial user input, aiming to broaden the search and find more relevant results (e.g., "educational toys for 10-year-olds," "adventure books for boys," "coding kits for kids").

In essence, "Generative Recommendation" moves beyond simple keyword matching to a more dynamic and intelligent approach where the AI proactively assists the user by generating new ideas and relevant search paths to help them discover desired products.

In the end, we will build an agent system that works in the following flow:

![Shop agent sequence diagram](../../../docs/assets/shop_agent.png)

<!-- Mermaid code for the diagram
sequenceDiagram
   participant User
   participant Shop Agent
   participant Research Agent
   participant Google Search
   participant find_shopping_items
   participant Vector Search

   User->>Shop Agent: Any birthday present for my son?
   activate Shop Agent
   Shop Agent->>Research Agent: Any birthday present for my son?
   activate Research Agent
   Research Agent->>Google Search: search on birthday presents for boys
   activate Google Search
   Google Search->>Research Agent: search results
   deactivate Google Search
   Research Agent->>Research Agent: generate 5 queries
   Research Agent->>Shop Agent: "STEM Toys", "Lego sets"...
   deactivate Research Agent
   Shop Agent->>User: Here are the suggested queries: "STEM Toys", "Lego sets"...
   Shop Agent->>User: Is it OK to search with those queries?
   deactivate Shop Agent

   activate Shop Agent
   User->>Shop Agent: Yes, please!
   activate find_shopping_items
   Shop Agent->>find_shopping_items: "STEM Toys", "Lego sets"...
   activate Vector Search
   find_shopping_items->>Vector Search: "STEM Toys", "Lego sets"...
   Vector Search->>find_shopping_items: "Kids Walkie talkies", "Minecraft Explorers Pack"...
   deactivate Vector Search
   find_shopping_items->>Shop Agent : "Kids Walkie talkies", "Minecraft Explorers Pack"...
   deactivate find_shopping_items
   Shop Agent->>User : "Kids Walkie talkies", "Minecraft Explorers Pack"...
   deactivate Shop Agent
-->

## Install ADK

First, we will install the ADK. In Colab Enterprise, you may see `ERROR: pip's dependency resolver does...` but you can ignore it.

In [ ]:
%pip install google-adk -q

### Import the libraries and set environment variables

In [ ]:
# Import necessary libraries
import os
import logging
import asyncio
from google.adk.agents import Agent
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.tools.agent_tool import AgentTool
from google.genai import types

# Ignore warnings from ADK and Gemini APIs
logging.getLogger("google.adk.runners").setLevel(logging.ERROR)
logging.getLogger("google_genai.types").setLevel(logging.ERROR)

# Set environment variables required for running ADK
[PROJECT_ID] = !gcloud config list --format "value(core.project)"
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["GOOGLE_CLOUD_LOCATION"] = "us-central1"
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "True"

## Define **test_agent** function for testing agents

For testing the agents we will build, we need to define a function `test_agent` that uses `Runner` and `SessionService` to emulate an agent runtime environment. To learn more about the agent runtime, see the [Agent Runtime](https://google.github.io/adk-docs/runtime/) doc.

In [ ]:
from google.genai import types

# Define the app_name, user_id and session_id for testing the agents
APP_NAME = "shop_concierge_app"
USER_ID = "user_1"
SESSION_ID = "session_001"

# Create a session
session_service = InMemorySessionService()
session = session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)

async def test_agent(query, agent):
  """Sends a query to the agent and prints the final response."""

  print(f"\n>>> User Query: {query}")

  # Create a Runner
  runner = Runner(
      agent=agent,
      app_name=APP_NAME,
      session_service=session_service
  )

  # Prepare the user's message in ADK format
  content = types.Content(role='user', parts=[types.Part(text=query)])

  final_response_text = None
  # We iterate through events from run_async to find the final answer.
  async for event in runner.run_async(user_id=USER_ID, session_id=SESSION_ID, new_message=content):
      if event.is_final_response():
          if event.content and event.content.parts:
             final_response_text = event.content.parts[0].text
          break
  print(f"<<< Agent Response: {final_response_text}")

## Define an **Shop agent**

Let's define a shop agent and test it. Note that this agent does not have any search capability at this time.

In [ ]:
instruction = f'''
    Your role is a shop search agent on an e-commerce site with millions of
    items. Your responsibility is to search items based on user queries.
'''

shop_agent = Agent(
    model='gemini-2.0-flash-001',
    name='shop_agent',
    description=(
        'Shop agent for an e-commerce site'
    ),
    instruction=instruction,
)

In [ ]:
await test_agent("What kind of site is this?", shop_agent)

## Define **call_vector_search** to call the Vector Search backend

With the basic agent above, we would like to add an item search capability. To achieve this, here we define a function `call_query_api` that sends an HTTP request to a REST endpoint provided by the [Vector Search Interactive demo](https://cloud.google.com/vertex-ai/docs/vector-search/try-it). For the detail of each parameter sent to the endpoint, refer to the demo page.

In [ ]:
import requests
import json

def call_vector_search(url, query, rows=None):
    """
    Calls the Vector Search backend for querying.

    Args:
        url (str): The URL of the search endpoint.
        query (str): The query string.
        rows (int, optional): The number of result rows to return. Defaults to None.

    Returns:
        dict: The JSON response from the API.
    """

    # Build HTTP headers and a payload
    headers = {'Content-Type': 'application/json'}
    payload = {
        "query": query,
        "rows": rows,
        "dataset_id": "mercari3m_mm", # Use Mercari 3M multimodal index
        "use_dense": True, # Use multimodal search
        "use_sparse": True, # Use keyword search too
        "rrf_alpha": 0.5, # Both results are merged with the same weights
        "use_rerank": True, # Use Ranking API for reranking
    }

    # Send an HTTP request to the search endpoint
    try:
        response = requests.post(url, headers=headers, data=json.dumps(payload))
        response.raise_for_status()  # Raise an exception for bad status codes
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error calling the API: {e}")
        return None

## Define **find_shopping_items** tool

Now, we will wrap the `call_vector_search` function with an ADK Tool named `find_shopping_items`. Note that we need to 1) use the explicit typing such as `queries: list[str]` and 2) use the verbose docstring, both for conveying the functionality and semantics of this Tool to the agent.

For details of the Tool mechanism of ADK, refer to the [Tools](https://google.github.io/adk-docs/tools/) in the ADK docs.

In [ ]:
from typing import Dict

def find_shopping_items(queries: list[str]) -> Dict[str, str]:
    """
    Find shopping items from the e-commerce site with the specified list of
    queries.

    Args:
        queries: the list of queries to run.
    Returns:
        A dict with the following one property:
            - "status": returns the following status:
                - "success": successful execution
            - "items": items found in the e-commerce site.
    """
    url = "https://www.ac0.cloudadvocacyorg.joonix.net/api/query"

    items = []
    for query in queries:
        result = call_vector_search(
            url=url,
            query=query,
            rows=3,
        )
        items.extend(result["items"])

    print("-----")
    print(f"User queries: {queries}")
    print(f"Found: {len(items)} items")
    print("-----")

    return items

Let's test this tool.

In [ ]:
find_shopping_items(["Cups with dancing people", "Cups with dancing animals"])

## Add the tool to **Shop agent**

It's ready to add the Tool to the Shop agent. The following parts are added:

- Addition to the `instruction`: `To find items use find_shopping_items tool by passing a list of queries, and answer to the user with item's name, description and img_url`
- Adding `tools` parameter to the `Agent` constructor: `tools=[find_shopping_items]`



In [ ]:
instruction = f'''
    Your role is a shop search agent on an e-commerce site with millions of
    items. Your responsibility is to search items based on the queries you
    recieve.

    To find items use `find_shopping_items` tool by passing a list of queries,
    and answer to the user with item's name, description and img_url
'''

shop_agent = Agent(
    model='gemini-2.0-flash-001',
    name='shop_agent',
    description=(
        'Shop agent for an e-commerce site'
    ),
    instruction=instruction,
    tools=[find_shopping_items],
)

Let's test the agent.

In [ ]:
await test_agent("Cups with dancing figures", shop_agent)

## Define **Research agent** with Google Search grounding

Next, we will define another agent `research_agent`. This agent will take the user query and use the built-in Google Search tool for researching on what kind of items people are purchasing for the user's intent. Then generate 5 queries for finding those items.

Note that the following agent definition specifies `google_search` as a tool. With this, the agent obtains a capability to use Google Search. For details about the Google Search tool, refer to [Google Search](https://google.github.io/adk-docs/tools/built-in-tools/#google-search) on the ADK docs.

In [ ]:
from google.adk.tools import google_search

instruction = f'''
    Your role is a market researcher for an e-commerce site with millions of
    items.

    When you recieved a search request from an user, use Google Search tool to
    research on what kind of items people are purchasing for the user's intent.

    Then, generate 5 queries finding those items on the e-commerce site and
    return them.
'''

research_agent = Agent(
    model='gemini-2.0-flash-001',
    name='research_agent',
    description=('''
        A market researcher for an e-commerce site. Receives a search request
        from a user, and returns a list of 5 generated queries in English.
    '''),
    instruction=instruction,
    tools=[google_search],
)

Let's test the agent.

In [ ]:
await test_agent("birthday present for 10 years old boy", research_agent)

## Finalize the **Shop agent**

To wrap up, we modify the `shop_agent` to use both `reseach_agent` and `find_shopping_items` tool.

In [ ]:
instruction = f'''
    Your role is a shopper's concierge for an e-commerce site with millions of
    items. Follow the following steps.

    1. Market research: When you recieved a search request from an user,
    pass it to `research_agent` tool, and receive 5 generated queries.
    Share the queries with the user, and ask if they want to continue.

    2. Find items: When use requested finding items with the queries, pass
    the list of queries to `find_shopping_items` to
    find items. When you recieved a list of items from the tool, answer to the
    user with item's name, description and the image url.
'''

shop_agent = Agent(
    model='gemini-2.0-flash-001',
    name='shop_agent',
    description=(
        'A shopper\'s concierge for an e-commerce site'
    ),
    instruction=instruction,
    tools=[
        AgentTool(agent=research_agent),
        find_shopping_items,
    ],
)

Let's test the agent. First, the user asks the agent for finding items. The Shop agent will call the Research agent for generating queries using Google Search Results.

In [ ]:
await test_agent("Can you find birthday present for 10 years old son?", shop_agent)

The user will reply "Yes" to let the agent proceed with the item search.

In [ ]:
await test_agent("Yes.", shop_agent)

## Summary

In this tutorial, we went through the process of constructing a multi-agent system for an e-commerce platform, focusing on "Generative Recommendations".

Through this progression, the tutorial illustrated how to build a sophisticated AI agent system that can understand user intent, perform research, generate targeted queries, and provide relevant product recommendations in an e-commerce context using ADK and external search capabilities.
